# Opioid distribution in California

This notebook documents the Los Angeles Times' analysis of U.S. Drug Enforcement Agency data released by The Washington Post in July 2019 as part of an [investigation of opioids distribution](https://www.washingtonpost.com/graphics/2019/investigations/dea-pain-pill-database/?utm_term=.9717a00308fe) nationwide. The data, pulled from the agency's Automation of Reports and Consolidated Order System, known as ARCOS, show prescription pain pill transactions from manufacurers to distributors from 2006 to 2012. During that time, more than 8 billion tablets of hydrocodone and oxycodone were distributed to California.

### To begin, load python libraries and set config params

In [ ]:
import pandas as pd
import psycopg2 as pg
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import rc
%matplotlib inline
import seaborn as sns
import sqlalchemy
import pandas.io.sql as psql
import numpy as np
import geopandas 

plt.rcParams['figure.figsize'] = [8, 4.5]
rc('font',**{'family':'sans-serif','sans-serif':['DejaVu Sans'],'size':10})
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

sorted(mpl.style.available)

### Connect to the local posgres server with California data

In [ ]:
database = pg.connect("dbname=opioids user=postgres")

### Query database to create dataframe for analysis with python and pandas

In [ ]:
pillssql = psql.read_sql("SELECT transaction_date, buyer_dea_no, buyer_bus_act, buyer_name, buyer_city, buyer_zip, buyer_county, drug_name, quantity, (case when length(dosage_unit) > 1 then dosage_unit end) as dosage_unit, mme_conversion_factor FROM opioidsca", database)

### To speed things up, try reading the from a tsv, not postgres

In [ ]:
pills = pd.read_csv("Desktop/data/arcos-ca-statewide-itemized.tsv", sep='\t', header=0, low_memory=False)

In [ ]:
pop = pd.read_csv("Desktop/data/ca-counties-pop-1970-2050.csv", sep=',', header=0, low_memory=False)

### Read the tsv with proper data types: first on import — and then after

In [ ]:
pills = pd.read_csv("Desktop/data/arcos-ca-statewide-itemized.tsv", sep='\t', header=0, low_memory=False, dtype={'BUYER_ZIP': 'str'})

In [ ]:
pills['REPORTER_ZIP'] = pills.REPORTER_ZIP.astype(object)

In [ ]:
pills['DRUG_CODE'] = pills.DRUG_CODE.astype(object)

### Transform the original int64 date field to a usable date

In [ ]:
pills['TRANSACTION_DATE'] = pd.to_datetime(pills['TRANSACTION_DATE'].astype('str'), format = '%m%d%Y')

### Parse the date further to extact year and month in new fields

In [ ]:
pills['TRANSACTION_DATE'] = pd.to_datetime(pills['TRANSACTION_DATE'])
pills['YEAR'], pills['MONTH'] = pills['TRANSACTION_DATE'].dt.year, pills['TRANSACTION_DATE'].dt.month

### Change month, year fields to objects for grouping

In [ ]:
pills['YEAR'] = pills.YEAR.astype(object)

In [ ]:
pills['MONTH'] = pills.MONTH.astype(object)

### A look at all the data types for each field in our data population frame

In [ ]:
pop['year'] = pop.year.astype(object)
pop['fips'] = pop.fips.astype(object)
pop.dtypes

### A look at all the data types for each field in our pills data frame

In [ ]:
pills.dtypes

In [ ]:
years = [2006,2007,2008,2009,2010,2011,2012]
popyears = pop[pop.year.isin(years)]

In [ ]:
# popyears.groupby(['year','county'])[['pop_total']].sum()
pillyears = popyears.groupby(['year','county'], as_index=False).agg({"pop_total": "sum"})

### How many people (should update to just adults) lived in each California county in each of the years from 2006 to 2012?

In [ ]:
pillyears

In [ ]:
pills['county'] = pills.BUYER_COUNTY.astype(object)

In [ ]:
pills

### Merge our tables for pill by county and population by county

In [ ]:
pillyears['year'] = pillyears.year.astype(object)
pillscounties['year'] = pillscounties.YEAR.astype(object)

In [ ]:
merged = pd.merge(pillyears,pillscounties, on=['county','year'])

In [ ]:
merged

### Create a rate field in our merged table

In [ ]:
merged['rate'] = merged['DOSAGE_UNIT']/merged['pop_total']

### New merged table with county, drug, business type, pills, population and rate — sorted desc by rate
### (These places are the outliers)

In [ ]:
merged.to_csv('Desktop/data.csv', sep='\t', encoding='utf-8')
print(merged)

In [ ]:
merged.sort_values(by='rate', ascending=False)

### First three rows in our dataframe

In [ ]:
pills.head(3)

### Buyers in LA County

In [ ]:
lapills = pills[pills.BUYER_COUNTY == 'LOS ANGELES']

### First three rows of LA pills dataframe

In [ ]:
lapills.head(3)

### Retail pharmarcies in the city of Los Angeles

In [ ]:
pills[(pills.BUYER_CITY == 'LOS ANGELES') & (pills.BUYER_BUS_ACT == 'RETAIL PHARMACY')]

### All transactions involving a buyer in zipcode 90066

In [ ]:
pills[(pills.BUYER_ZIP == "90066")][['BUYER_NAME','BUYER_ZIP', 'BUYER_COUNTY', 'TRANSACTION_DATE']]

In [ ]:
pills['DOSAGE_UNIT'].sum()

In [ ]:
mpl.style.use('seaborn-poster')

In [ ]:
def doplot():
    pills['BUYER_ZIP'].value_counts()[:10].plot(kind='barh')

mpl.style.use('ggplot')
doplot()

In [ ]:
pills.head(5)

### How many pills by type were sold in Los Angeles County over the years? 

In [ ]:
pills[pills.BUYER_COUNTY == 'LOS ANGELES'].groupby(['YEAR', 'DRUG_NAME'])[('DOSAGE_UNIT')].sum()

### How did distribution of Oxycodone to California change over the time period? 

In [ ]:
def doplot():
    pills[(pills.DRUG_NAME == 'OXYCODONE')].groupby(['YEAR'])[('DOSAGE_UNIT')].sum().plot(kind='barh')

mpl.style.use('ggplot')
doplot()

### What's the percentage change of oxycodone in CA during this time? 

In [ ]:
pillscounties = pills.groupby(['YEAR','BUYER_COUNTY', 'BUYER_BUS_ACT','DRUG_NAME'], as_index=False).agg({"DOSAGE_UNIT": "sum"})

In [ ]:
pillscounties['county'] = pillscounties.BUYER_COUNTY.astype(object)

In [ ]:
pillscounties

### How did distribution of oxycodone to retail pharmacies change in the LA Times' zipcode over time? 

In [ ]:
def doplot():
    pills[(pills.BUYER_ZIP == '90066') & (pills.DRUG_NAME == 'OXYCODONE') & (pills.BUYER_BUS_ACT == 'RETAIL PHARMACY')].groupby(['YEAR'])[('DOSAGE_UNIT')].sum().plot.barh(y='YEAR')

mpl.style.use('ggplot')
doplot()

### Who were the largest distributors in CA during the period?

### How does pill distribution in California compare, per capita, with other states?

### How does pill distribution in Los Angeles County compare, per capita, with other counties?

### Which CA cities had the highest per capita pill distribution?

### Oxy distribution to retail pharmacies in zipcode 90066 over time?

In [ ]:
def doplot():
    pills[(pills.BUYER_ZIP == '90066') & (pills.DRUG_NAME == 'OXYCODONE') & (pills.BUYER_BUS_ACT == 'RETAIL PHARMACY')].groupby(['YEAR'])[('DOSAGE_UNIT')].sum().plot.barh(y='YEAR')
    
mpl.style.use('ggplot')
doplot()

In [ ]:
camap = geopandas.read_file(/Users/mhustiles/Desktop/data/GIS/ca-counties.geojson)

In [ ]:
camap